In [9]:
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# import data
kreise_df = pd.read_csv('/home/jan/Uni/DS-Project/modules/dashboard/test/assets/kreise_df.csv')
# rename and drop columns
kreise_df = kreise_df.drop(columns=['Unnamed: 0', 'ID_1', 'ID_3']).rename(columns={'NAME_1': 'Bundesland', 'NAME_3': 'Landkreis', 'ENGTYPE_3': 'Land_Stadt'})

# good if there are many options
Bundesland_unique = kreise_df['Bundesland'].unique()
Bundesland_options = [{'label': item, 'value': item} for item in np.sort(kreise_df['Bundesland'].unique())]

In [8]:
# create sunburst plot
Ohren_sunburst = px.sunburst(kreise_df, path=['Bundesland', 'Landkreis'], values='avg_Score', color='Land_Stadt')
print(pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/718417069ead87650b90472464c7565dc8c2cb1c/coffee-flavors.csv').head())
Ohren_sunburst.show()
kreise_df.head()

                       ids    labels parents
0        Enzymatic-Flowery   Flowery     NaN
1         Enzymatic-Fruity    Fruity     NaN
2          Enzymatic-Herby     Herby     NaN
3     Sugar Browning-Nutty     Nutty     NaN
4  Sugar Browning-Carmelly  Carmelly     NaN


,Bundesland,Landkreis,Land_Stadt,Zahl_Ohren,Zahl_PV,avg_Score
0,Niedersachsen,Oldenburg,Rural district,26,278,0.56
1,Niedersachsen,Osnabrück Städte,Urban district,9,94,4.03
2,Niedersachsen,Osnabrück,Rural district,10,211,4.50
3,Niedersachsen,Vechta,Rural district,22,471,9.30
4,Niedersachsen,Wesermarsch,Rural district,22,240,3.55


In [10]:
df = kreise_df

ids = df['Bundesland'] + '-' + df['Landkreis']
labels = df['Landkreis']

fig = go.Figure()

fig.add_trace(go.Sunburst(
    ids=ids,
    labels=labels,
    parents=df['Bundesland'],
    values=df['avg_Score'],
    maxdepth=2,
    insidetextorientation='radial'
))

fig.update_layout(
    margin = dict(t=10, l=10, r=10, b=10)
)

fig.show()

In [3]:
app = Dash(__name__, external_stylesheets = [dbc.themes.LITERA])

In [4]:
# change to app.layout if running as single page app instead
app.layout = html.Div([
    dbc.Container([
        # Überschriften & Einleitung
        dbc.Row([
            dbc.Col(html.H1("Lustnauer 'Ohren' skaliert auf ganz Deutschland", className="text-center")
                    , className="mb-5 mt-5")
        ]),
        dbc.Row([
            dbc.Col(html.H5(children='Erkennen von ungenutzten Potenzialen in ungenutzten Flächen.'
                                     )
                    , className="mb-4")
            ]),

        dbc.Row([
            dbc.Col(html.H5(children='Auf der zweiten Seite wird eine interaktive Karte dargestellt, welche die verschiedenen Ohren ranked'
                                     'Die dritte Seite präsentiert dieselben Daten in tabellarischer Form, zum erkunden und nachschlagen.')
                    , className="mb-5")
        ]),

        # Sunburst Plot
        dbc.Row([
            dcc.Graph(figure = Ohren_sunburst)
        ]),
        
        # Links und Erklärungen
        dbc.Row([

            dbc.Col(dbc.Card(children=[html.H3(children='GitHub Repository',
                                               className="text-center"),
                                       dbc.Button("GitHub",
                                                  href="https://github.com/felixschulz385/ds_project",
                                                  color="primary",
                                                  className="mt-3"),
                                       ],
                             body=True, color="dark", outline=True)
                    , width=4, className="mb-4"),

            dbc.Col(dbc.Card(children=[html.H3(children='Wie alles begann',
                                               className="text-center"),
                                       dbc.Button("Zeitungsartikel",
                                                  href="https://www.badische-zeitung.de/ob-palmer-will-solaranlagen-auf-freien-stellen-neben-bundesstrassen--211487616.html",
                                                  color="primary",
                                                  className="mt-3"),

                                       ],
                             body=True, color="dark", outline=True)
                    , width=4, className="mb-4")
        ], className="mb-5")

    ])

])

In [5]:
# run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [05/Feb/2023 17:32:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 17:32:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 17:32:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 17:32:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2023 17:32:08] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


# Test Bereich